In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from time import sleep 
%matplotlib qt

In [9]:

def regionGrow(img, seeds, thresh, diffType,label):
    height, weight = img.shape
    seedMark = np.zeros(img.shape)
    seedList = []
    steps = 1
    seeds_grayscale = []
    seeds_avg_grayscale = 0

    for seed in seeds:
        seedList.append(seed)
        seeds_grayscale.append(img[seed[0], seed[1]])
    seeds_avg_grayscale = np.average(seeds_grayscale)
    
    connects = [ (0, -1), (1, 0),(0, 1), (-1, 0)]
    while(len(seedList)>0):
        currentPoint = seedList.pop(0)
        if diffType=='static':
            seedMark[currentPoint[0], currentPoint[1]] = label
        
            for i in range(4):
                tmpX = currentPoint[0] + connects[i][0]
                tmpY = currentPoint[1] + connects[i][1]
                if tmpX < 0 or tmpY < 0 or tmpX >= height or tmpY >= weight:
                    continue

                grayDiff = abs (img[tmpX, tmpY] - seeds_avg_grayscale)

                if grayDiff < thresh and seedMark[tmpX,tmpY] == 0:
                    seedMark[tmpX,tmpY] = label
                    seedList.append((tmpX,tmpY))


        if (diffType == "dynamic"):
            seeds_grayscale.append(img[currentPoint[0], currentPoint[1]])
            seeds_avg_grayscale = np.average(seeds_grayscale)
        seedMark[currentPoint[0], currentPoint[1]] = label
        
        for i in range(4):
            tmpX = currentPoint[0] + connects[i][0]
            tmpY = currentPoint[1] + connects[i][1]
            if tmpX < 0 or tmpY < 0 or tmpX >= height or tmpY >= weight:
                continue

            grayDiff = abs (img[tmpX, tmpY] - seeds_avg_grayscale)

            if grayDiff < thresh and seedMark[tmpX,tmpY] == 0:
                seedMark[tmpX,tmpY] = label
                seedList.append((tmpX,tmpY))

        steps +=1   


    print ("Number of steps: ", steps)
    return seedMark

In [10]:
MRI_img = cv.imread('Color_MRI.png', 0)
white_seed_coor=[(385,344)]
gray_seed_coor=[(190,467)]
thresh_type = 'static'
empty_img=np.zeros(MRI_img.shape)
empty_img[(385,344)]=255
empty_img[(190,467)]=100
thresh_w = 70
thresh_g = 30
seedMark_white = np.uint8(regionGrow(MRI_img, white_seed_coor, thresh_w, thresh_type,255))
seedMark_gray= np.uint8(regionGrow(MRI_img, gray_seed_coor, thresh_g, thresh_type,100))
seedmark_final = np.uint8(seedMark_gray + seedMark_white)

plt.figure('show seeds')
plt.imshow(empty_img,'gray',vmin=0,vmax=255)
plt.figure('show results')
plt.subplot(1,2,1)
plt.imshow(MRI_img,  cmap='gray', vmin = 0, vmax = 255)
plt.title('original')
plt.axis('off')
plt.subplot(1,2,2)

plt.imshow(seedmark_final,  cmap='gray', vmin = 0, vmax = 255)
plt.axis('off')
plt.title('after segmentation')
plt.tight_layout()
plt.show()



Number of steps:  100998
Number of steps:  114339
